In [1]:
import tensorflow.compat.v1 as tf
import numpy as np
from scipy.stats import norm

C:\Anaconda3\envs\tensorflow\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda3\envs\tensorflow\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda3\envs\tensorflow\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda3\envs\tensorflow\lib\site-packages\tensorboard\compat\tensorf

In [2]:
import matplotlib.pyplot as plt
from equation import PricingOption, AllenCahn, HJB
from config import PricingOptionConfig, AllenCahnConfig, HJBConfig

import time
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization

In [3]:
TF_DTYPE = tf.float32

class FeedForwardModel(object):

    def __init__(self, sess, bsde, config):
        self._sess = sess
        self.config = config
        self.bsde = bsde
        
        self.dim = bsde._dim
        self.total_time = bsde._total_time
        self.num_time_interval = bsde._num_time_interval
        self.delta_t = bsde._delta_t
                
        self.f_network = []
        self.z_network=[]
    
    
    def test(self):
        self.start_time = time.time()
    
        dw_valid,x_valid = self.bsde.sample(self.config.valid_size)
        feed_dict_valid = {self._x: x_valid,self._dw:dw_valid,
                           self.lambda1:1,self.lambda2:0, 
                           self._is_training: False}
        
        feed_dict_valid = {self._x: x_valid, self._dw:dw_valid, 
                           self.lambda1: 1, self.lambda2: 2, self._is_training: False}
        self._sess.run(tf.global_variables_initializer())
        
        for step in range(self.config.num_iterations + 1):
            if step % self.config.logging_frequency == 0:
                loss, init = self._sess.run([self._loss, self._y_init], feed_dict=feed_dict_valid)
                elapsed_time = time.time() - self.start_time + self._t_build
                print("step: %5u,loss: %.4e,y_init: %.4e  elapsed time %3u" % (step, loss, init, elapsed_time))
            dw_train,x_train = self.bsde.sample(self.config.batch_size)
            loss=self._sess.run([self._loss ,self._train_ops], 
                                feed_dict={self._x: x_train,self._dw:dw_train,self.lambda1:0,
                                           self.lambda2:1,self._is_training: True})[0]
   
    def train(self):
        self.start_time = time.time()
    
        dw_valid,x_valid = self.bsde.sample(self.config.valid_size)
        
        feed_dict_valid = {self._x: x_valid, self._dw:dw_valid, 
                           self.lambda1: 1, self.lambda2: 0, 
                           self._is_training: False}
        self._sess.run(tf.global_variables_initializer())
        for step in range(self.config.num_iterations + 1):
            if step % self.config.logging_frequency == 0:
                loss, init = self._sess.run([self._loss, self._y_init], feed_dict=feed_dict_valid)
                elapsed_time = time.time() - self.start_time + self._t_build
                print("step: %5u,loss: %.4e,y_init: %.4e  elapsed time %3u" % (step, loss, init, elapsed_time))
            dw_train,x_train = self.bsde.sample(self.config.batch_size)
            loss=self._sess.run([self._loss ,self._train_ops], 
                                feed_dict={self._x: x_train,self._dw:dw_train,self.lambda1:1,
                                           self.lambda2:0,self._is_training: True})[0]
        
        # calculate upper bound:
        dw_valid,x_valid = self.bsde.sample(10000)
        feed_dict_valid = {self._x: x_valid, self._dw:dw_valid, 
                           self.lambda1: 0, self.lambda2: 1, 
                           self._is_training: False}
        
        for step in range(self.config.num_iterations + 1):
            dw_train,x_train = self.bsde.sample(self.config.batch_size)
            loss=self._sess.run([self._loss ,self._train_ops], 
                                feed_dict={self._x: x_train,self._dw:dw_train,self.lambda1:0,
                                           self.lambda2:1,self._is_training: True})[0]
            if step % self.config.logging_frequency == 0:
                loss, init = self._sess.run([self._loss, self._y_init], feed_dict=feed_dict_valid)
                elapsed_time = time.time() - self.start_time + self._t_build
                print("step: %5u,upper_bound: %.4e , elapsed time %3u" % (step, np.sqrt(loss), elapsed_time))
        
        
        
        
    def nn_structure(self, _network, _input):
        z = _network[0](_input)
        for i in range(1, len(_network)):
            z = _network[i](z)
        return z
    
    def relu_adding(self, network, unit_lst, activation = None):
        #num >=2
        num = len(unit_lst)
        network.append(Dense(units = unit_lst[0], input_shape = (self.dim,), activation = 'relu'))
        for i in range(1, num):
            network.append(Dense(units = unit_lst[i], input_shape = (unit_lst[i-1],), activation = 'relu'))
        network.append(Dense(units = self.dim, input_shape = (unit_lst[-1],), activation = activation))
    
    def build(self):
        start_time = time.time()
        time_stamp = np.arange(0, self.num_time_interval) * self.delta_t
        
        for i in range(self.num_time_interval-1):
            temp = []
            temp.append(BatchNormalization())
            self.relu_adding(temp, self.config.z_units)
            self.z_network.append(temp) 
        
        self._x = tf.placeholder(TF_DTYPE, [None,self.dim, self.num_time_interval+1], name='X')
        self._dw = tf.placeholder(TF_DTYPE, [None, self.dim, self.num_time_interval], name='dW')
        self.lambda1=tf.placeholder(TF_DTYPE, name='lambda1')
        self.lambda2=tf.placeholder(TF_DTYPE, name='lambda2')
        self._is_training = tf.placeholder(tf.bool)
        
        self._y_init = tf.Variable(tf.random_uniform([1],
                                                     minval=self.config.y_init_range[0],
                                                     maxval=self.config.y_init_range[1],
                                                     dtype=TF_DTYPE))
        self._z_init = tf.Variable(tf.random_uniform([1],
                                               minval=-.1, maxval=.1,
                                               dtype=TF_DTYPE))
        with tf.variable_scope('forward'):
            
            all_one_vec = tf.ones(shape=tf.stack([tf.shape(self._dw)[0], 1]), dtype=TF_DTYPE)
            y = all_one_vec * self._y_init
            z = all_one_vec * self._z_init
            
            for t in range(0, self.num_time_interval - 1):
                y = y - self.delta_t * (
                    self.bsde.f_tf(time_stamp[t], self._x[:, :, t], y, z))
                
                y = y + tf.reduce_sum(z * self.bsde._sigma
                * self._dw[:, :, t], 1, keepdims=True)
                
                z = self.nn_structure(self.z_network[t], self._x[:, :, t + 1])

            y = y - self.bsde.delta_t * self.bsde.f_tf(
                time_stamp[-1], self._x[:, :, -2], y, z)
            y = y + tf.reduce_sum(z * self.bsde._sigma 
                * self._dw[:, :, -1], 1, keepdims=True)
            
            loss1 = y - self.bsde.g_tf(self.total_time, self._x[:, :, -1])
            
            # the dual problem structure
            u=self.bsde.g_tf(self.total_time, self._x[:, :, -1])
            
            for t in range(self.num_time_interval - 1,0,-1):
                
                z=self.nn_structure(self.z_network[t-1], self._x[:, :, t])
                
                u=u+self.delta_t * (
                    self.bsde.f_tf(time_stamp[t], self._x[:, :, t], u, z))
                
                u=u-tf.reduce_sum(z * self.bsde._sigma 
                    * self._dw[:, :, t], 1, keepdims=True)
            
            z = all_one_vec * self._z_init
            u=u+self.delta_t * (
                    self.bsde.f_tf(time_stamp[0], self._x[:, :, 0], u, z))
                
            u=u-tf.reduce_sum(z * self.bsde._sigma 
                    * self._dw[:, :, 0], 1, keepdims=True)
            
            loss2 = u+0.0

            self._loss = self.lambda1 * tf.reduce_mean(tf.square(loss1))+\
                            self.lambda2 * tf.square(tf.reduce_mean(loss2))

        trainable_variables = tf.trainable_variables()
        grads = tf.gradients(self._loss, trainable_variables)
        
        global_step = tf.get_variable('global_step', [],
                                      initializer=tf.constant_initializer(0),
                                      trainable=False, dtype=tf.int32)
        learning_rate = tf.train.piecewise_constant(global_step,
                                                    self.config.lr_boundaries,
                                                    self.config.lr_values)
        
        optimizer = tf.train.AdamOptimizer(learning_rate= learning_rate)

        apply_op = optimizer.apply_gradients(zip(grads, trainable_variables), global_step=global_step,name='train_step')
        all_ops = [apply_op]
        self._train_ops = tf.group(*all_ops)
        self._t_build = time.time()-start_time

In [4]:
# 4.4
#params
dim, total_time, num_time_interval = 5, 0.5, 10

#fit
Option= PricingOption(dim, total_time, num_time_interval)
tf.reset_default_graph()
with tf.Session() as sess:
    model = FeedForwardModel(sess, Option, PricingOptionConfig())
    model.build()
    model.train()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
step:     0,loss: 5.4601e+01,y_init: 5.7161e+00  elapsed time  11
step:  1000,loss: 3.0198e+01,y_init: 4.5802e+00  elapsed time  29
step:  2000,loss: 2.9698e+01,y_init: 4.5929e+00  elapsed time  46
step:  3000,loss: 2.5915e+01,y_init: 4.5561e+00  elapsed time  62
step:  4000,loss: 2.5799e+01,y_init: 4.6024e+00  elapsed time  77
step:  5000,loss: 2.4926e+01,y_init: 4.6139e+00  elapsed time  94
step:     0,upper_bound: 4.6696e+00 , elapsed time  94
step:  1000,upper_bound: 4.6626e+00 , elapsed time 110
step:  2000,upper_bound: 4.6704e+00 , elapsed time 127
step:  3000,upper_bound: 4.7006e+00 , elapsed time 144
step:  4000,upper_bound: 4.6724e+00 , elapsed time 162
step:  5000,upper_bound: 4.6769e+00 , elapsed time 180
